In [58]:
import numpy as np
import random
from sys import stdout
import difflib
from IPython.core.display import display, HTML, clear_output
from NeuralNetwork import NeuralNetwork as NN

In [59]:
k=0

In [60]:
nn = NN(2, [4], 2)
answerkey = [True,False]

In [61]:
answerkey[nn.predict([0.8,0.1]).argmax()]

False

In [62]:
out1 = str(nn)


In [63]:
data = [{'target':[0.1,0.8], 'input':[0.1,0.1]},
        {'target':[0.8,0.1], 'input':[0.1,0.8]},
        {'target':[0.8,0.1], 'input':[0.8,0.1]},
        {'target':[0.1,0.8], 'input':[0.8,0.8]}
       ]

In [65]:
i = 0
while i < 1000:
    clear_output()
    out1 = str(nn)
    i += 1
    random.shuffle(data)
    for d in data:
        nn.train(d['input'],d['target'])
    out2 = str(nn)
    display(HTML(difflib.HtmlDiff().make_file(out1.splitlines(), out2.splitlines())))
        

f,1,Weight Matrix[0]:,f,1,Weight Matrix[0]:
n,2,[[ 0.00835525 0.30520378],n,2,[[ 0.00837357 0.30519275]
,3,[-0.39042607 0.36004184],,3,[-0.39039835 0.36003067]
,4,[ 0.38059141 0.60977308],,4,[ 0.38057617 0.60978135]
,5,[ 0.69728932 0.08171399]],,5,[ 0.6972745 0.08172615]]
,6,Weight Matrix[1]:,,6,Weight Matrix[1]:
n,7,[[ 0.52391041 0.45420109 -0.37342131 -0.00741789],n,7,[[ 0.52390255 0.45419356 -0.37342351 -0.00742361]
,8,[ 0.23268892 0.54710308 -0.31228958 -0.45183836]],,8,[ 0.23268854 0.54709678 -0.31229697 -0.45183496]]
,9,Bias[0]:,,9,Bias[0]:
n,10,[[-0.27047657],n,10,[[-0.27048301]
,11,[-0.23766108],,11,[-0.23766614]


In [54]:
k += i
print('{} cycles'.format(k))
print(nn.predict([0.1,0.1]), not answerkey[nn.predict([0.1,0.1]).argmax()] )
print(nn.predict([0.1,0.8]), answerkey[nn.predict([0.1,0.8]).argmax()] )
print(nn.predict([0.8,0.1]), answerkey[nn.predict([0.8,0.1]).argmax()] )
print(nn.predict([0.8,0.8]), not answerkey[nn.predict([0.8,0.8]).argmax()] )

1 cycles
[[0.79191721]
 [0.83905774]] True
[[0.81228304]
 [0.85725593]] False
[[0.82630538]
 [0.85826272]] False
[[0.84183525]
 [0.87269509]] True


In [55]:
display(HTML(difflib.HtmlDiff().make_file(out1.splitlines(), out2.splitlines())))

f,1,Weight Matrix[0]:,f,1,Weight Matrix[0]:
n,2,[[0.41668093 0.67641791],n,2,[[0.97338624 0.32034879]
,3,[0.54528649 0.98268803],,3,[0.37290404 0.96325865]
,4,[0.87985056 0.31549883],,4,[0.24238458 0.28002881]
,5,[0.59583529 0.13766675]],,5,[0.02853613 0.81054729]]
,6,Weight Matrix[1]:,,6,Weight Matrix[1]:
n,7,[[0.31107519 0.29388114 0.60630997 0.88349267],n,7,[[0.28359191 0.121989 0.27322075 0.61534623]
,8,[0.63841369 0.30754138 0.49629201 0.08327426]],,8,[0.82304921 0.09827237 0.64384756 0.30726216]]
,9,Bias[0]:,,9,Bias[0]:
n,10,[[0.16285843],n,10,[[0.39717212]
,11,[0.42789907],,11,[0.95459307]


In [67]:
difflib.HtmlDiff().make_table(out1.splitlines(), out2.splitlines())

'\n    <table class="diff" id="difflib_chg_to1026__top"\n           cellspacing="0" cellpadding="0" rules="groups" >\n        <colgroup></colgroup> <colgroup></colgroup> <colgroup></colgroup>\n        <colgroup></colgroup> <colgroup></colgroup> <colgroup></colgroup>\n        \n        <tbody>\n            <tr><td class="diff_next" id="difflib_chg_to1026__0"><a href="#difflib_chg_to1026__0">f</a></td><td class="diff_header" id="from1026_1">1</td><td nowrap="nowrap">Weight&nbsp;Matrix[0]:</td><td class="diff_next"><a href="#difflib_chg_to1026__0">f</a></td><td class="diff_header" id="to1026_1">1</td><td nowrap="nowrap">Weight&nbsp;Matrix[0]:</td></tr>\n            <tr><td class="diff_next" id="difflib_chg_to1026__1"><a href="#difflib_chg_to1026__1">n</a></td><td class="diff_header" id="from1026_2">2</td><td nowrap="nowrap">[[&nbsp;0.00835<span class="diff_chg">525</span>&nbsp;&nbsp;0.3052<span class="diff_sub">03</span>7<span class="diff_chg">8</span>]</td><td class="diff_next"><a href="